## Importing necessary libraries

In [1]:
import pandas as pd
import math
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestRegressor

## Loading the transposed merged dataframe

In [2]:
df=pd.read_csv("processed_1.csv")
df=df.drop(["Unnamed: 0"],axis=1)
dff=df
df

,PatientIDWoundNumber,FinalVisit_Healing_Status,AccidentType,Anterior_Posterior,Anterior_Dorsal,DateInactive,DateOfAccident,DateWoundHealed,Inferior_Superior,Left_Right,...,TunnelingOrUndermining_Visit18,Width_Visit18,woundProgress_Visit18,WoundMargin_Visit18,eschar_Visit18,Wound_healing_status_Visit18,Wound_healing_percentage_Visit18,DayNumber_Visit18,DateAcquired_2,DateAdded_wound_2
0,0,0.0,0,0,2,1574,568,1586,2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,442,5681
1,2,3.0,0,2,1,1574,568,1586,2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2461,5324
2,3,3.0,0,2,1,1574,568,1586,1,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,457,3037
3,4,0.0,0,2,2,1574,568,1586,2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,917,5325
4,5,2.0,0,2,2,1574,568,1586,2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2524,5317
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19923,24432,1.0,0,2,2,1574,568,1586,2,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,264,3864
19924,24433,3.0,0,1,2,1574,568,1586,2,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,855,34981
19925,24434,3.0,0,0,2,1574,568,1586,2,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,250,2887
19926,24435,0.0,0,0,2,1574,568,1586,2,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,250,2872


In [3]:
# def drop_null_columns(df, threshold):
#     # Calculate the threshold for null values
#     null_threshold = len(df) * threshold
    
#     # Get the count of null values in each column
#     null_counts = df.isnull().sum()
    
#     # Create a list of columns to drop
#     columns_to_drop = null_counts[null_counts > null_threshold].index.tolist()
    
#     # Drop the columns from the dataframe
#     df_dropped = df.drop(columns=columns_to_drop)
    
#     return df_dropped

# # Example usage
# threshold=0.95
# df_dropped = drop_null_columns(df, threshold)
# df_dropped

In [4]:
df_dropped=df

In [5]:
df_dropped["FinalVisit_Healing_Status"].value_counts()

FinalVisit_Healing_Status
3.0    8343
1.0    6037
2.0    2967
0.0    2581
Name: count, dtype: int64

In [6]:
# Create an instance of LabelEncoder
label_encoder = LabelEncoder()

# Iterate over each column in the dataframe
for column in df_dropped.columns:
    print(column)
    # Check if the column is of object or categorical type
    if df_dropped[column].dtype == 'object':
        # Convert the column to string type
        df_dropped[column] = df_dropped[column].astype(str)
        
        # Fit and transform the categorical column
        df_dropped[column] = label_encoder.fit_transform(df_dropped[column])

PatientIDWoundNumber
FinalVisit_Healing_Status
AccidentType
Anterior_Posterior
Anterior_Dorsal
DateInactive
DateOfAccident
DateWoundHealed
Inferior_Superior
Left_Right
Location
Notes
Proximal_Distal
ResolvedRecurrence
ResultOfAccident
Type
WoundCondition
woundstatus
AdmissionDate
AdmissionStatusCode
MedicareAdmission
NonWoundDiagnosis
PalliativeCare
StartDate
CauseOfDeath
DateOfDeath
ethnicityCdcCode
PatientDOB
PatientGender
PatientSelfCare
PatientZIP
RaceCodeList
Religion
DateAdded_patient
Age
Height
Weight
Patient_RiskLevel
Patient_Score
BMI
SmokingStatus
SmokingStatusText
DebridementLevel
Method
Bleeding
IsThisBurn
SInsuranceZip
PInsuranceZip
AdiposeExposed_Visit1
AdiposeNecrosisExposed_Visit1
area_Visit1
AtrophieBlanche_Visit1
BrawnyInduration_Visit1
Callus_Visit1
color_Visit1
CompCompression_Visit1
CompDiet_Visit1
Crepitus_Visit1
Denuded_Visit1
Depth_Visit1
DryScaly_Visit1
Ecchymosis_Visit1
Edema_Visit1
Erythema_Visit1
Excoriation_Visit1
ExposedMuscle_Visit1
ExposedTendon_Visit1
E

In [7]:
# df_dropped = df_dropped.fillna(-99999)
# df_dropped=df_dropped.replace([np.inf, -np.inf], -99999)
# df_dropped

In [8]:
#determine the pattern of missingness


## Removing highly correlated features

In [13]:
from scipy.stats import pointbiserialr
import pingouin as pg

# Example dataset (replace this with your actual dataset)
data = df_dropped

# Custom correlation function to handle NaN values
def custom_correlation(x, y, method):
    valid_values = ~np.isnan(x) & ~np.isnan(y)
    x_valid = x[valid_values]
    y_valid = y[valid_values]
    
    if method == 'pearson':
        # Handle NaN values for Pearson correlation
        mask = np.isfinite(x_valid) & np.isfinite(y_valid)
        correlation_result = pointbiserialr(x_valid[mask], y_valid[mask])
    elif method == 'pointbiserial':
        correlation_result = pointbiserialr(x_valid, y_valid)
    else:
        raise ValueError(f"Unsupported correlation method: {method}")
    
    return correlation_result

# Create an empty dictionary to store correlation results
correlation_results = {}

# Iterate through numerical columns
for numerical_var in data.select_dtypes(include=['float64', 'int64']).columns:
    # Iterate through all columns
    for column in data.columns:
        # Skip the numerical variable itself
        if column == numerical_var:
            continue
        
        # Determine data types
        data_type_numerical = data[numerical_var].dtype
        data_type_column = data[column].dtype
        
        # Apply the appropriate correlation method based on data types
        if data_type_numerical in ['float64', 'int64'] and data_type_column in ['float64', 'int64']:
            # Both variables are numerical - use Pearson correlation
            correlation_result = custom_correlation(data[numerical_var], data[column], 'pearson')
        elif data_type_numerical in ['float64', 'int64'] and data_type_column == 'object':
            # Numerical variable and categorical variable - use point-biserial correlation
            # Convert binary categorical data to numeric
            data_numeric = pd.Categorical(data[column]).codes
            correlation_result = custom_correlation(data[numerical_var], data_numeric, 'pointbiserial')
        # Add other cases as needed
        else:
            # Other combinations - skip or handle accordingly
            continue
        
        # Store the correlation result
        key = f"{numerical_var}_vs_{column}"
        correlation_results[key] = correlation_result

# Print the results
for key, result in correlation_results.items():
    print(f"Correlation {key}:\n{result}\n")


/home/jupyter-apaddo/.local/lib/python3.9/site-packages/scipy/stats/_stats_py.py:4781: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [15]:
df=df_dropped

target_column='FinalVisit_Healing_Status'

X = df.drop([target_column, 'PatientIDWoundNumber'], axis=1)
y = df[target_column]

# Fit Random Forest regressor
rf = LGBMClassifier()
rf.fit(X, y)

for key, result in correlation_results.items():
    #print(key)
    # Split the string into two parts
    split_parts = key.split('_vs_')
    # Display the result
    
    
    if round(result[0],4)>0.8:
        
        print(split_parts)
        print("Correlation Coeff:",round(result[0],4))
        feature1=split_parts[0]
        feature2=split_parts[1]

        columns_to_check = [feature1, feature2]

        # Iterate through the columns and check if each column is present
        if set(columns_to_check).issubset(df.columns):
            print("Both columns are present in the DataFrame.")

            # Print feature importance
            print(f"\nFeature Importance for {feature1}: {pd.Series(rf.feature_importances_, index=X.columns)[feature1]}")
            print(f"Feature Importance for {feature2}: {pd.Series(rf.feature_importances_, index=X.columns)[feature2]}")

            feature_1_importance=pd.Series(rf.feature_importances_, index=X.columns)[feature1]
            feature_2_importance=pd.Series(rf.feature_importances_, index=X.columns)[feature2]

            # Drop the less important feature
            if feature_1_importance < feature_2_importance:
                df.drop(feature1, axis=1, inplace=True)
                print("Dropping",feature1,"\n")
            else:
                df.drop(feature2, axis=1, inplace=True)
                print("Dropping",feature2,"\n")

    # Display the modified DataFrame
display(df)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.026528 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 34280
[LightGBM] [Info] Number of data points in the train set: 19928, number of used features: 1193
[LightGBM] [Info] Start training from score -2.043949
[LightGBM] [Info] Start training from score -1.194219
[LightGBM] [Info] Start training from score -1.904574
[LightGBM] [Info] Start training from score -0.870703
['AdmissionStatusCode', 'MedicareAdmission']
Correlation Coeff: 0.8159
Both columns are present in the DataFrame.

Feature Importance for AdmissionStatusCode: 34
Feature Importance for MedicareAdmission: 48
Dropping AdmissionStatusCode 

['MedicareAdmission', 'AdmissionStatusCode']
Correlation Coeff: 0.8159
['AdiposeExposed_Visit1', 'AdiposeExposed_Visit2']
Correlation Coeff: 0.8266
Both columns are present in the DataFrame.

,PatientIDWoundNumber,FinalVisit_Healing_Status,AccidentType,Anterior_Posterior,Anterior_Dorsal,DateInactive,DateOfAccident,DateWoundHealed,Inferior_Superior,Left_Right,...,Stage_Visit18,TunnelingOrUndermining_Visit18,Width_Visit18,woundProgress_Visit18,eschar_Visit18,Wound_healing_status_Visit18,Wound_healing_percentage_Visit18,DayNumber_Visit18,DateAcquired_2,DateAdded_wound_2
0,0,0.0,0,0,2,1574,568,1586,2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,442,5681
1,2,3.0,0,2,1,1574,568,1586,2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2461,5324
2,3,3.0,0,2,1,1574,568,1586,1,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,457,3037
3,4,0.0,0,2,2,1574,568,1586,2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,917,5325
4,5,2.0,0,2,2,1574,568,1586,2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2524,5317
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19923,24432,1.0,0,2,2,1574,568,1586,2,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,264,3864
19924,24433,3.0,0,1,2,1574,568,1586,2,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,855,34981
19925,24434,3.0,0,0,2,1574,568,1586,2,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,250,2887
19926,24435,0.0,0,0,2,1574,568,1586,2,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,250,2872


In [17]:
df

,PatientIDWoundNumber,FinalVisit_Healing_Status,AccidentType,Anterior_Posterior,Anterior_Dorsal,DateInactive,DateOfAccident,DateWoundHealed,Inferior_Superior,Left_Right,...,Stage_Visit18,TunnelingOrUndermining_Visit18,Width_Visit18,woundProgress_Visit18,eschar_Visit18,Wound_healing_status_Visit18,Wound_healing_percentage_Visit18,DayNumber_Visit18,DateAcquired_2,DateAdded_wound_2
0,0,0.0,0,0,2,1574,568,1586,2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,442,5681
1,2,3.0,0,2,1,1574,568,1586,2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2461,5324
2,3,3.0,0,2,1,1574,568,1586,1,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,457,3037
3,4,0.0,0,2,2,1574,568,1586,2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,917,5325
4,5,2.0,0,2,2,1574,568,1586,2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2524,5317
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19923,24432,1.0,0,2,2,1574,568,1586,2,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,264,3864
19924,24433,3.0,0,1,2,1574,568,1586,2,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,855,34981
19925,24434,3.0,0,0,2,1574,568,1586,2,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,250,2887
19926,24435,0.0,0,0,2,1574,568,1586,2,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,250,2872


In [16]:
# target_column='FinalVisit_Healing_Status'

# # Set the threshold for correlation
# threshold = 0.90

# # Chunk size
# chunk_size = 500

# # List to store reduced DataFrames
# dfs_reduced = []

# # Process the DataFrame in chunks
# for start in range(2, len(df_dropped.columns), chunk_size):
#     end = start + chunk_size
#     print("column:",start,"to column:",end)
#     df_chunk = df_dropped.iloc[:, start:end]

#     df=df_chunk
    
#     # Calculate correlation matrix
#     #corr_matrix = df.drop(target_column, axis=1).corr()
#     correlation_matrix = df.corr()

#     # Step 2: Identify Highly Correlated Features
#     threshold = 0.95
#     highly_correlated_pairs = []

#     for i in range(len(df.columns)):
#         for j in range(i + 1, len(df.columns)):
#             if abs(correlation_matrix.iloc[i, j]) > threshold:
#                 highly_correlated_pairs.append((df.columns[i], df.columns[j]))

#     # Step 3 and 4: Evaluate Feature Importance and Drop Less Important Features
#     for feature1, feature2 in highly_correlated_pairs:
#         # Create a subset DataFrame with the correlated features and the target feature
#         if all(feature in df.columns for feature in [feature1, feature2]):
#             subset_df = pd.concat([df[[feature1, feature2]],df_dropped[target_column]],axis=1)

#             # Prepare data for Random Forest
#             X = subset_df[[feature1, feature2]]
#             y = subset_df[target_column]

#             # Fit Random Forest regressor
#             rf = LGBMClassifier()
#             rf.fit(X, y)

#             # Print feature importance
#             print(f"\nFeature Importance for {feature1}: {rf.feature_importances_[0]}")
#             print(f"Feature Importance for {feature2}: {rf.feature_importances_[1]}")

#             # Drop the less important feature
#             if rf.feature_importances_[0] < rf.feature_importances_[1]:
#                 df.drop(feature1, axis=1, inplace=True)
#                 print("Dropping",feature1,"\n")
#             else:
#                 df.drop(feature2, axis=1, inplace=True)
#                 print("Dropping",feature2,"\n")

#     # Display the modified DataFrame
#     display(df)
    
#     # Append the reduced DataFrame to the list
#     dfs_reduced.append(df)

# # Concatenate the reduced DataFrames
# df_reduced = pd.concat(dfs_reduced, axis=1)

# # Display the final reduced DataFrame
# df_reduced


In [11]:
# df_reduced=pd.concat([df_dropped[['PatientIDWoundNumber','FinalVisit_Healing_Status']],df_reduced],axis=1)
# df_reduced

In [13]:
# import pandas as pd
# import numpy as np
# from scipy.stats import pearsonr

# # Set the threshold for correlation
# threshold = 0.95

# # Chunk size
# chunk_size = 500

# df_new=df_reduced
# # List to store reduced DataFrames
# dfs_reduced = []

# # Process the DataFrame in chunks
# for start in range(0, len(df_new.columns), chunk_size):
#     end = start + chunk_size
#     print("column:",start,"to column:",end)
#     df_chunk = df_dropped.iloc[:, start:end]

#     # Calculate the correlation matrix for the chunk
#     correlation_matrix = df_chunk.corr().abs()

#     # Find pairs of highly correlated features
#     highly_correlated_features = set()
#     for i in range(len(correlation_matrix.columns)):
#         for j in range(i):
#             if abs(correlation_matrix.iloc[i, j]) > 0.95:  # Adjust the threshold as needed
#                 colname = correlation_matrix.columns[i]
#                 highly_correlated_features.add(colname)

#     # Create a dictionary to store correlations with the target variable
#     correlation_with_target = {}

#     # Iterate through pairs of highly correlated features
#     for feature in highly_correlated_features:
#         # Calculate correlation of each feature with the target variable
#         correlation_with_target[feature] = df_chunk[feature].corr(df_dropped['FinalVisit_Healing_Status'])

#     # Remove the feature with lower correlation with the target variable
#     for feature, corr in correlation_with_target.items():
#         other_feature = [f for f in highly_correlated_features if f != feature][0]
#         other_corr = correlation_with_target[other_feature]

#         if df_chunk.columns.isin([feature]).any() and abs(corr) < abs(other_corr):
#             # Remove the feature with lower correlation
#             df_chunk = df_chunk.drop(columns=[feature])
#         elif df_chunk.columns.isin([other_feature]).any():
#             # Remove the other feature
#             df_chunk = df_chunk.drop(columns=[other_feature])
    
    
#     # Find and drop highly correlated columns within the chunk
#     #upper_triangle = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool))
#     #to_drop = [column for column in upper_triangle.columns if any(upper_triangle[column] > threshold)]
#     #df_chunk_reduced = df_chunk.drop(to_drop, axis=1)
#     df_chunk_reduced=df_chunk
    
#     # Append the reduced DataFrame to the list
#     dfs_reduced.append(df_chunk_reduced)

# # Concatenate the reduced DataFrames
# df_reduced = pd.concat(dfs_reduced, axis=1)

# # Display the final reduced DataFrame
# df_reduced

In [14]:
# for m in range(0,5):    
#     # Set the threshold for correlation
#     threshold = 0.90

#     # Chunk size
#     chunk_size = 500

#     df_new=df_reduced
#     # List to store reduced DataFrames
#     dfs_reduced = []

#     # Process the DataFrame in chunks
#     for start in range(0, len(df_new.columns), chunk_size):
#         end = start + chunk_size
#         print("column:",start,"to column:",end)
#         df_chunk = df_dropped.iloc[:, start:end]

#         # Calculate the correlation matrix for the chunk
#         correlation_matrix = df_chunk.corr().abs()

#         # Find pairs of highly correlated features
#         highly_correlated_features = set()
#         for i in range(len(correlation_matrix.columns)):
#             for j in range(i):
#                 if abs(correlation_matrix.iloc[i, j]) > 0.90:  # Adjust the threshold as needed
#                     colname = correlation_matrix.columns[i]
#                     highly_correlated_features.add(colname)

#         # Create a dictionary to store correlations with the target variable
#         correlation_with_target = {}

#         # Iterate through pairs of highly correlated features
#         for feature in highly_correlated_features:
#             # Calculate correlation of each feature with the target variable
#             correlation_with_target[feature] = df_chunk[feature].corr(df_dropped['FinalVisit_Healing_Status'])

#         # Remove the feature with lower correlation with the target variable
#         for feature, corr in correlation_with_target.items():
#             other_feature = [f for f in highly_correlated_features if f != feature][0]
#             other_corr = correlation_with_target[other_feature]

#             if df_chunk.columns.isin([feature]).any() and abs(corr) < abs(other_corr):
#                 # Remove the feature with lower correlation
#                 df_chunk = df_chunk.drop(columns=[feature])
#             elif df_chunk.columns.isin([other_feature]).any():
#                 # Remove the other feature
#                 df_chunk = df_chunk.drop(columns=[other_feature])


#         # Find and drop highly correlated columns within the chunk
#         #upper_triangle = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool))
#         #to_drop = [column for column in upper_triangle.columns if any(upper_triangle[column] > threshold)]
#         #df_chunk_reduced = df_chunk.drop(to_drop, axis=1)
#         df_chunk_reduced=df_chunk

#         # Append the reduced DataFrame to the list
#         dfs_reduced.append(df_chunk_reduced)

#     # Concatenate the reduced DataFrames
#     df_reduced = pd.concat(dfs_reduced, axis=1)

#     # Display the final reduced DataFrame
#     display(df_reduced)

## Removing features using feature importance

In [15]:
# from sklearn.ensemble import RandomForestClassifier
# import pandas as pd

# target_column = 'FinalVisit_Healing_Status'

# # Assuming df_reduced is your original DataFrame
# X = df_reduced.drop([target_column, 'PatientIDWoundNumber'], axis=1)
# y = df_reduced[target_column]

# # Create a Random Forest classifier
# rf = RandomForestClassifier()

# # Fit the model
# rf.fit(X, y)

# # Get the feature importances
# feature_importances = pd.Series(rf.feature_importances_, index=X.columns)

# # Sort the feature importances in descending order
# feature_importances = feature_importances.sort_values(ascending=False)

# # Filter out columns with 0 or less feature importance
# selected_columns = feature_importances[feature_importances > 0.0001].index

# # Create a new DataFrame with selected columns and target variable
# df_selected = pd.concat([X[selected_columns], y], axis=1)

# # Print the feature importances
# print(feature_importances)

# # Print the new DataFrame with selected columns and target variable
# df_selected

In [16]:
# df=df_dropped
# target_column='FinalVisit_Healing_Status'
# df=df.replace([np.inf, -np.inf], -99999)

# # Calculate correlation matrix
# corr_matrix = df.drop(target_column, axis=1).corr()

# # Create boolean mask for highly correlated features
# threshold = 0.95
# mask = np.abs(corr_matrix) > threshold
# np.fill_diagonal(mask.values, False)  # Exclude self-correlation

# # Calculate feature importances using Random Forest classifier
# X = df.drop([target_column,"PatientIDWoundNumber"], axis=1)  # Features
# y = df[target_column]  # Target variable

# rf = RandomForestClassifier()
# rf.fit(X, y)
# feature_importances = pd.Series(rf.feature_importances_, index=X.columns)

# # Combine the masked correlation matrix and feature importances
# combined_df = pd.concat([feature_importances, mask.any()], axis=1, keys=['Importance', 'HighlyCorrelated'])

# # Sort the combined DataFrame by feature importance values
# combined_df = combined_df.sort_values(by='Importance', ascending=False)

# # Select features based on importance and correlation
# selected_features = []
# for feature in combined_df.index:
#     if not combined_df.loc[feature, 'HighlyCorrelated']:
#         selected_features.append(feature)

# # Add the target variable column back to the selected features
# selected_features.append(target_column)

# # Create the filtered dataframe with selected features
# df_filtered = df[selected_features]

# df_filtered

In [18]:
df_reduced=df

In [19]:
df_reduced.to_csv("df_reduced.csv")

In [20]:
#pip install lightgbm --install-option=--gpu

In [21]:
df_reduced=pd.read_csv("df_reduced.csv")
df_reduced=df_reduced.drop(["Unnamed: 0"],axis=1)
df_reduced

,PatientIDWoundNumber,FinalVisit_Healing_Status,AccidentType,Anterior_Posterior,Anterior_Dorsal,DateInactive,DateOfAccident,DateWoundHealed,Inferior_Superior,Left_Right,...,Stage_Visit18,TunnelingOrUndermining_Visit18,Width_Visit18,woundProgress_Visit18,eschar_Visit18,Wound_healing_status_Visit18,Wound_healing_percentage_Visit18,DayNumber_Visit18,DateAcquired_2,DateAdded_wound_2
0,0,0.0,0,0,2,1574,568,1586,2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,442,5681
1,2,3.0,0,2,1,1574,568,1586,2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2461,5324
2,3,3.0,0,2,1,1574,568,1586,1,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,457,3037
3,4,0.0,0,2,2,1574,568,1586,2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,917,5325
4,5,2.0,0,2,2,1574,568,1586,2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2524,5317
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19923,24432,1.0,0,2,2,1574,568,1586,2,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,264,3864
19924,24433,3.0,0,1,2,1574,568,1586,2,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,855,34981
19925,24434,3.0,0,0,2,1574,568,1586,2,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,250,2887
19926,24435,0.0,0,0,2,1574,568,1586,2,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,250,2872


In [22]:
prefixes_to_convert = ['Method', 'SmokingStatus', 'NonWoundDiagnosis', 'PatientGender',
    'MedicareAdmission', 'SmokingStatusText', 'WoundCondition', 'Location',
    'AdmissionStatusCode', 'Religion', 'Depth', 'Pain', 'GranulationTexture',
    'ExudateAmount', 'NormalColor', 'PalliativeCare', 'Patient_RiskLevel',
    'color', 'Exudate', 'Type', 'RedGranulation', 'Left_Right', 'PatientDOB',
    'WoundMargin', 'tunneling', 'RaceCodeList', 'PatientSelfCare', 'PInsuranceZip',
    'Bleeding', 'surroundingSkin', 'surroundingSkinColor', 'Anterior_Dorsal',
    'CauseOfDeath', 'Proximal_Distal', 'woundstatus', 'ActiveWound',
    'Inferior_Superior', 'normalTexture', 'ResolvedRecurrence', 'Stage',
    'slough', 'IsThisBurn', 'woundProgress', 'Notes', 'CompCompression',
    'SloughLessThan20', 'Anterior_Posterior', 'Odor', 'eschar',
    'TunnelingOrUndermining', 'SSXRay', 'Patient_Title', 'NormalMoisture',
    'ResolvedRecurrence', 'Stage', 'IsThisBurn', 'CompCompression',
    'SloughLessThan20', 'Anterior_Posterior', 'Odor', 'eschar',
    'TunnelingOrUndermining', 'SSXRay', 'Patient_Title', 'NormalMoisture',
    'ResolvedRecurrence','FinalVisit_Healing_Status','PatientZIP']

# Iterate over columns
for col in df_reduced.columns:
    for prefix in prefixes_to_convert:
        if col.startswith(prefix) or col == prefix:
            df_reduced[col] = df_reduced[col].astype('object')

# Verify the changes
print(df_reduced.dtypes)

PatientIDWoundNumber                  int64
FinalVisit_Healing_Status            object
AccidentType                          int64
Anterior_Posterior                   object
Anterior_Dorsal                      object
                                     ...   
Wound_healing_status_Visit18        float64
Wound_healing_percentage_Visit18    float64
DayNumber_Visit18                   float64
DateAcquired_2                        int64
DateAdded_wound_2                     int64
Length: 467, dtype: object


In [24]:
# from imblearn.over_sampling import SMOTE
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
# from sklearn.preprocessing import StandardScaler
# from sklearn.metrics import classification_report, accuracy_score

# target_column = 'FinalVisit_Healing_Status'

# # Split the data into features (X) and the target variable (y)
# X = df_reduced.drop([target_column], axis=1)
# y = df_reduced[target_column]

# print("Unique values in target variable:", y.unique())
# print("Data type of the target variable:", y.dtype)
# # Convert target variable to integer type
# y = y.astype(int)


# # Split the data into training and test sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Apply SMOTE to the training data
# smote = SMOTE(random_state=42)
# X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# # Standardize features
# scaler = StandardScaler()
# X_train_resampled_scaled = scaler.fit_transform(X_train_resampled)
# X_test_scaled = scaler.transform(X_test)

# #total_instances = len(y)
# #class_weights = {0: total_instances / 2581, 1: total_instances / 6037, 2: total_instances / 2967, 3: total_instances / 8343}



# rf = RandomForestClassifier(random_state=42)
# #grid_search = GridSearchCV(rf, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
# grid_search=rf
# grid_search.fit(X_train_resampled_scaled, y_train_resampled)

# # # Get the best model from the grid search
# # best_rf = grid_search.best_estimator_
# best_rf = grid_search

# # Get feature importances
# feature_importances = best_rf.feature_importances_

# # Identify features with non-zero importance
# important_feature_indices = [i for i, importance in enumerate(feature_importances) if importance > 0.0005]

# # Filter the features based on importance
# X_train_resampled_scaled_important = X_train_resampled_scaled[:, important_feature_indices]
# X_test_scaled_important = X_test_scaled[:, important_feature_indices]

# # Train the model on the important features
# best_rf_important = RandomForestClassifier(random_state=42)
# best_rf_important.fit(X_train_resampled_scaled_important, y_train_resampled)

# # Predict the target variable for the test set
# y_pred_important = best_rf_important.predict(X_test_scaled_important)

# # Generate the classification report
# report_important = classification_report(y_test, y_pred_important)
# print(report_important)

# # Accuracy on the test set
# accuracy_important = accuracy_score(y_test, y_pred_important)
# print("Test Set Accuracy (Important Features):", accuracy_important)


In [27]:
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score

df_reduced=df_reduced.replace([np.inf, -np.inf], np.nan)


target_column = 'FinalVisit_Healing_Status'

# Split the data into features (X) and the target variable (y)
X = df_reduced.drop([target_column], axis=1)
y = df_reduced[target_column]

print("Unique values in target variable:", y.unique())
print("Data type of the target variable:", y.dtype)
# Convert target variable to integer type
y = y.astype(int)


# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Apply SMOTE to the training data
# smote = SMOTE(random_state=42)
# X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

X_train_resampled, y_train_resampled = X_train, y_train

# Standardize features
scaler = StandardScaler()
X_train_resampled_scaled = scaler.fit_transform(X_train_resampled)
X_test_scaled = scaler.transform(X_test)

total_instances = len(y)
class_weights = {0: total_instances / 2581, 1: total_instances / 6037, 2: total_instances / 2967, 3: total_instances / 8343}

rf = LGBMClassifier(random_state=42)
#grid_search = GridSearchCV(rf, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search=rf
grid_search.fit(X_train_resampled_scaled, y_train_resampled)

# # Get the best model from the grid search
# best_rf = grid_search.best_estimator_
best_rf = grid_search

# Get feature importances
feature_importances = best_rf.feature_importances_

# Identify features with non-zero importance
important_feature_indices = [i for i, importance in enumerate(feature_importances) if importance > 5]

# Filter the features based on importance
X_train_resampled_scaled_important = X_train_resampled_scaled[:, important_feature_indices]
X_test_scaled_important = X_test_scaled[:, important_feature_indices]

# Train the model on the important features
best_rf_important = LGBMClassifier(random_state=42)
best_rf_important.fit(X_train_resampled_scaled_important, y_train_resampled)

# Predict the target variable for the test set
y_pred_important = best_rf_important.predict(X_test_scaled_important)

# Generate the classification report
report_important = classification_report(y_test, y_pred_important)
print(report_important)

# Accuracy on the test set
accuracy_important = accuracy_score(y_test, y_pred_important)
print("Test Set Accuracy (Important Features):", accuracy_important)


Unique values in target variable: [0. 3. 2. 1.]
Data type of the target variable: float64
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005520 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 19119
[LightGBM] [Info] Number of data points in the train set: 15942, number of used features: 461
[LightGBM] [Info] Start training from score -2.048195
[LightGBM] [Info] Start training from score -1.192662
[LightGBM] [Info] Start training from score -1.901017
[LightGBM] [Info] Start training from score -0.871787
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002678 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 18213
[LightGBM] [Info] Number of data points in the train set: 15942, number of use

In [30]:
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score

target_column = 'FinalVisit_Healing_Status'

# Split the data into features (X) and the target variable (y)
X = df_reduced.drop([target_column], axis=1)
y = df_reduced[target_column]

print("Unique values in the target variable:", y.unique())
print("Data type of the target variable:", y.dtype)
# Convert target variable to integer type
y = y.astype(int)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Apply SMOTE to the training data
# smote = SMOTE(random_state=42)
# X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

X_train_resampled, y_train_resampled = X_train, y_train

# Standardize features
scaler = StandardScaler()
X_train_resampled_scaled = scaler.fit_transform(X_train_resampled)
X_test_scaled = scaler.transform(X_test)

# Define XGBoost classifier
xgb_classifier = XGBClassifier(random_state=42)

# Fit the model on resampled and scaled data
xgb_classifier.fit(X_train_resampled_scaled, y_train_resampled)

# Get feature importances
feature_importances = xgb_classifier.feature_importances_

# Identify features with non-zero importance
important_feature_indices = [i for i, importance in enumerate(feature_importances) if importance > 0]

# Filter the features based on importance
X_train_resampled_scaled_important = X_train_resampled_scaled[:, important_feature_indices]
X_test_scaled_important = X_test_scaled[:, important_feature_indices]

# Train the model on the important features
xgb_classifier_important = XGBClassifier(random_state=42)
xgb_classifier_important.fit(X_train_resampled_scaled_important, y_train_resampled)

# Predict the target variable for the test set
y_pred_important = xgb_classifier_important.predict(X_test_scaled_important)

# Generate the classification report
report_important = classification_report(y_test, y_pred_important)
print(report_important)

# Accuracy on the test set
accuracy_important = accuracy_score(y_test, y_pred_important)
print("Test Set Accuracy (Important Features):", accuracy_important)


Unique values in the target variable: [0. 3. 2. 1.]
Data type of the target variable: float64
              precision    recall  f1-score   support

           0       0.98      0.86      0.91       525
           1       0.93      0.93      0.93      1200
           2       0.98      0.90      0.94       585
           3       0.90      0.96      0.93      1676

    accuracy                           0.93      3986
   macro avg       0.95      0.91      0.93      3986
weighted avg       0.93      0.93      0.93      3986

Test Set Accuracy (Important Features): 0.9295032614149523


In [31]:
X.columns[important_feature_indices].tolist()

['PatientIDWoundNumber',
 'AccidentType',
 'Anterior_Posterior',
 'DateOfAccident',
 'Inferior_Superior',
 'Left_Right',
 'Location',
 'Notes',
 'Proximal_Distal',
 'ResolvedRecurrence',
 'ResultOfAccident',
 'Type',
 'WoundCondition',
 'AdmissionDate',
 'MedicareAdmission',
 'NonWoundDiagnosis',
 'PalliativeCare',
 'StartDate',
 'CauseOfDeath',
 'DateOfDeath',
 'ethnicityCdcCode',
 'PatientDOB',
 'PatientGender',
 'PatientSelfCare',
 'PatientZIP',
 'RaceCodeList',
 'Religion',
 'DateAdded_patient',
 'Age',
 'Height',
 'Weight',
 'Patient_RiskLevel',
 'Patient_Score',
 'BMI',
 'SmokingStatus',
 'SmokingStatusText',
 'DebridementLevel',
 'Method',
 'Bleeding',
 'IsThisBurn',
 'SInsuranceZip',
 'PInsuranceZip',
 'AdiposeExposed_Visit1',
 'AdiposeNecrosisExposed_Visit1',
 'area_Visit1',
 'Callus_Visit1',
 'color_Visit1',
 'Depth_Visit1',
 'DryScaly_Visit1',
 'Erythema_Visit1',
 'Excoriation_Visit1',
 'Exudate_Visit1',
 'ExudateAmount_Visit1',
 'Friable_Visit1',
 'imageFileCount_Visit1',
 

In [33]:
import numpy as np
import pandas as pd
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, LSTM

# Assuming df_reduced, target_column, X, y, X_train, X_test, y_train, y_test are already defined
target_column = 'FinalVisit_Healing_Status'

# Split the data into features (X) and the target variable (y)
X = df_reduced.drop([target_column], axis=1)
y = df_reduced[target_column]

print("Unique values in target variable:", y.unique())
print("Data type of the target variable:", y.dtype)
# Convert target variable to integer type
y = y.astype(int)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features (optional but can be beneficial for some models)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Reshape the data for RNN input
# The input shape for an RNN is (batch_size, timesteps, input_features)
X_train_reshaped = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test_reshaped = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

# Build the RNN model
model = Sequential()
model.add(LSTM(units=500, input_shape=(1, 466), activation='relu', return_sequences=True))  # Adjust units and activation as needed
model.add(LSTM(units=500, activation='sigmoid',return_sequences=True))
model.add(LSTM(units=500, activation='sigmoid'))
model.add(Dense(units=1, activation='sigmoid'))  # For binary classification

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_reshaped, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test_reshaped, y_test)
print(f'Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}')

Unique values in target variable: [0. 3. 2. 1.]
Data type of the target variable: float64
Epoch 1/10
399/399 [==============================] - 14s 32ms/step - loss: nan - accuracy: 0.1309 - val_loss: nan - val_accuracy: 0.1214
Epoch 2/10
399/399 [==============================] - 12s 31ms/step - loss: nan - accuracy: 0.1309 - val_loss: nan - val_accuracy: 0.1214
Epoch 3/10
399/399 [==============================] - 12s 31ms/step - loss: nan - accuracy: 0.1309 - val_loss: nan - val_accuracy: 0.1214
Epoch 4/10
399/399 [==============================] - 12s 31ms/step - loss: nan - accuracy: 0.1309 - val_loss: nan - val_accuracy: 0.1214
Epoch 5/10
399/399 [==============================] - 12s 31ms/step - loss: nan - accuracy: 0.1309 - val_loss: nan - val_accuracy: 0.1214
Epoch 6/10
399/399 [==============================] - 12s 31ms/step - loss: nan - accuracy: 0.1309 - val_loss: nan - val_accuracy: 0.1214
Epoch 7/10
399/399 [==============================] - 12s 31ms/step - loss: nan - 

In [11]:
# 3 = not healing
# 0 = grey
# 2 = healing
# 1 = healed

In [ ]:
# -SMOTE
# -Use class weights for training